<a href="https://colab.research.google.com/github/AV-BOLT/Used_car_price_prediction/blob/master/car_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание проекта #


Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение, чтобы привлечь новых клиентов. В нём можно будет узнать рыночную стоимость своего автомобиля.

Постройте модель, которая умеет её определять. В вашем распоряжении данные о технических характеристиках, комплектации и ценах других автомобилей.

**Критерии, которые важны заказчику:**
- качество предсказания;
- время обучения модели;
- время предсказания модели.

Инструкция по выполнению проекта

Основные шаги:
Загрузите данные, путь к файлу: /datasets/autos.csv.
Изучите данные. Заполните пропущенные значения и обработайте аномалии в столбцах. Если среди признаков имеются неинформативные, удалите их.
Подготовьте выборки для обучения моделей.
Обучите разные модели, одна из которых — LightGBM, как минимум одна — не бустинг.

Для каждой модели попробуйте разные гиперпараметры.
Проанализируйте время обучения, время предсказания и качество моделей.

Опираясь на критерии заказчика, выберете лучшую модель, проверьте её качество на тестовой выборке.


Описание данных
Данные находятся в файле autos.csv.

Признаки
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- Repaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя
- Целевой признак Price — цена (евро)



Критерии, которые важны заказчику:
- качество предсказания;
- время обучения модели;
- время предсказания модели.

**Требования:**

Обучите разные модели, одна из которых
- LightGBM,
- как минимум одна — не бустинг.
- Для каждой модели попробуйте разные гиперпараметры.
- Проанализируйте **время обучения**, **время предсказания** и к**ачество моделей**.
-Опираясь на критерии заказчика, выберете лучшую модель, проверьте её качество на тестовой выборке.

**Примечания:**

- Для оценки качества моделей применяйте метрику RMSE.
Значение метрики RMSE должно быть меньше 2500.
- Самостоятельно освойте библиотеку LightGBM и её средствами постройте модели градиентного бустинга.

# Загрузка данных и библиотек

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import seaborn as sns
#import optuna

In [2]:
import time

In [3]:
!pip install catboost
import catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.8 MB/s eta 0:00:00


In [4]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data = pd.read_csv('/content/drive/MyDrive/Projects_YP/Used_car_price_prediction/autos.csv')
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [7]:
data.shape

(354369, 16)

Данные загружены, имеется 354 369 объектов и 16 признаков.


# Предобработка и анализ

In [8]:
# посмотрим на дынные
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [9]:
data.dtypes.to_frame()

,0
DateCrawled,object
Price,int64
VehicleType,object
RegistrationYear,int64
Gearbox,object
Power,int64
Model,object
Kilometer,int64
RegistrationMonth,int64
FuelType,object


- В данных имеются пропуски
- Имеются признаки с типом данных который не соответствует значениям в признаке
- Названия необходимо привести к единообразной форме (строчный регистр и _ для разделения)
- Данные представлены категориальными, числовыми признаками, datetime.


Этапы предобработки данных:

**приведение типов данных** к требуемому формату datetime (без времени):
- DateCrawled
- DateCreated
- LastSeen

**приведение названия признаков**  к единообразному формату

**пропуски**:
- Анализ причин возникновения пропусков
- Рассмотрение возможных способов обработки таких признаков(заглушка, использование inputer, оставить как есть если позволяет модель, заполнение синтетическими данными, заполнение на основании остальных признаков и т.д.)

примерный алгоритм обработки **категориальные признаки**,  значит необходимо обрабатывать их - приведение к единому написанию (регистр, пробелы/символы/цифры и тд.)
- проверка на дупликаты (явные и неявные)

**Числовые признаки** проверим
- на аномалии и редкие значения
-



## Приведение значений к требуемому формату

In [10]:
# Посмотрим, если они в одном формате представлены
#то их одновременно приведем к формату datetime
data[['DateCrawled', 'DateCreated', 'LastSeen']].head()

,DateCrawled,DateCreated,LastSeen
0,2016-03-24 11:52:17,2016-03-24 00:00:00,2016-04-07 03:16:57
1,2016-03-24 10:58:45,2016-03-24 00:00:00,2016-04-07 01:46:50
2,2016-03-14 12:52:21,2016-03-14 00:00:00,2016-04-05 12:47:46
3,2016-03-17 16:54:04,2016-03-17 00:00:00,2016-03-17 17:40:17
4,2016-03-31 17:25:20,2016-03-31 00:00:00,2016-04-06 10:17:21


In [11]:
date_columns = ['DateCrawled', 'DateCreated', 'LastSeen']

In [12]:
data[date_columns] = data[date_columns]\
            .applymap(lambda x: pd.to_datetime(x).date())

data[date_columns].head()

,DateCrawled,DateCreated,LastSeen
0,2016-03-24,2016-03-24,2016-04-07
1,2016-03-24,2016-03-24,2016-04-07
2,2016-03-14,2016-03-14,2016-04-05
3,2016-03-17,2016-03-17,2016-03-17
4,2016-03-31,2016-03-31,2016-04-06


Проверим, не одинаковые ли значения содержат признаки 'DateCrawled' и 'DateCreated', если разница будет в районе одного месяца то можно уверенно удалить один из признаков.

In [13]:
(data['DateCrawled'] == data['DateCreated']).all()


False

Значения в этих признаках не совпадают, значит они не дублирующие друг друга.

Оценим разницу в этих признаках

In [ ]:
date_difference = data['DateCrawled'] - data['DateCreated']

In [ ]:
date_difference.describe().to_frame()

Видим, что большая часть пользователей заполняют анкеты в день скачивания,

In [ ]:
date_difference.value_counts(ascending = False).head(10).to_frame().transpose()

In [ ]:
date_difference.value_counts(ascending = True).head(10).to_frame().transpose()

  Несмотря на то, что в этих данных имеются различия, мы можем заменить признак date_crawled на признак date_difference.
  Это сделать целесобразно потому что этот признак мы сохраним в числовом формате, что удобнее для кодирования и обработки.

In [ ]:
data['date_difference'] = date_difference.dt.days
data['date_difference'].head()

In [ ]:
data['date_difference'].dtype

- Из значений в признаках, содержащих дату со временем извлечена дата, что позволит работать нам с только с датой
- Создан новый признак date_difference, который содержит количество дней от дня скачивания до дня создания анкеты.

## Приведение названий признаков к змеиному формату

In [ ]:
data.columns

Необходимо:
- привести к строчному регистру
- добавить _ между словами

In [ ]:
data.columns = ['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gear_box',
       'power', 'model', 'kilometer', 'registration_month', 'fuel_type', 'brand',
       'repaired', 'date_created', 'pictures_amount', 'postal_code',
       'last_seen','date_difference']

In [ ]:
data.columns

Названия исправлены.

## Анализ пропущенных значений

Заполнять все пропуски я буду заглушками, которые отличимы от реальных данных.

Синтетические данные я подбирать не буду.

Для каждого признака с пропущенными значениями я построю график чтобы оценить какие значения содержатся и какой масштаб (для числовых признаков).

- Числовые признаки числом, отличимым по масштабу от реальных данных -1
- Категориальные признаки заглушкой **no_info**

In [ ]:
def pass_value_barh(df):
    try:(
        (df.isna().mean()*100)
            .to_frame()
            .rename(columns = {0:'space'})
            .query('space > 0')
            .sort_values(by = 'space', ascending = True)
            .plot(kind = 'barh',
                  figsize = (19,6),
                  rot = -0,
                  legend = False,
                  fontsize = 16,
                  xticks=[i/10 for i in range(0, 1000, 50)],
                  color='skyblue')
            .set_title('Количество пропущенных значений' + "\n", fontsize = 22, color = 'Black')
        );
    except:
        print('пропусков не осталось')

In [ ]:
pass_value_barh(data)

### repaired

In [ ]:
data['repaired'].value_counts(dropna=False).plot(kind='bar',
                                                 color='skyblue',
                                                 figsize = (6,2))
plt.title('Распределение числа ремонтированных тс')
plt.ylabel('Количество')
plt.xticks(rotation=45)
plt.show()

### vehicle_type

In [ ]:
data['vehicle_type'].value_counts(dropna=False).plot(kind='barh',
                                                     color='skyblue',
                                                     figsize = (8,4))
plt.title('Распределение Типов Транспортных Средств')
plt.ylabel('Тип Транспортного Средства')
plt.xlabel('Количество')
plt.tight_layout()
plt.show()

### fuel_type

In [ ]:
data['fuel_type'].value_counts(dropna=False).plot(kind='barh',
                                                  color='skyblue',
                                                  figsize = (8,2))
plt.title('Распределение Типов топлива')
plt.ylabel('Тип топлива')
plt.xlabel('Количество')
plt.tight_layout()
plt.show()

### gear_box

In [ ]:
data['gear_box'].value_counts(dropna=False).plot(kind='barh',
                                                  color='skyblue',
                                                  figsize = (8,2))
plt.title('Распределение Типов коробки передач')
plt.ylabel('Тип коробки')
plt.xlabel('Количество')
plt.tight_layout()
plt.show()

### model

следующий признак один из важнейших, но график строить нецелесообразно из-за того, что в данном признаке содержится слишком много уникальных названий моделей автомобилей.

Значения категориальные и следовательно заполняем заглушкой no_info

### Заполняем пропуски

In [ ]:
data = data.fillna('no_info')

In [ ]:
pass_value_barh(data)

In [ ]:
data.head()


Пропуски имеются исключительно в категориальных признаках.

Пропуски заполнены.

## Анализ числовых признаков на аномалии

In [ ]:
data.dtypes

Для анализа распределения воспользуемся графиками.

Для этого заранее подготовим переменные в которых сохраним названия признаков содержащих одинаковый тип данных.

In [ ]:
categorical_columns = data.select_dtypes(include = 'object').columns
numeric_columns = data.select_dtypes(include = 'number').columns

In [ ]:
# Оценим визуально признаки, их распределение
data[numeric_columns].hist(layout=(4,4),figsize=(15, 20));

- price нужно изучить на наличие аномалий
- в признаке registration_year очевидно имееются аномалии
- признак power нужно проверить на систему в которой выражена мощность.
- kilometer имеются редкие значения, но аномалий не видно на этом графике
- pictures_amount возможно содержат отрицательные значения, что является аномалией
- sale_month, postal_code в обработке и анализе не нуждаются

#### registration_year

Начнем с этого признака, так как там несомненные аномалии

In [ ]:
data[['registration_year']].hist();

In [ ]:
data[['registration_year']].plot(
    kind = 'hist',
    range = (int(data[['registration_year']].min()), 1950),
    bins = 50,
    xticks=[x for x in range(1000, 1970, 50)],
    rot = -45,
    figsize = (12,6)
);
plt.title ('Распределение признака registration_year до 1950 года');

In [ ]:
data[['registration_year']].plot(
    kind = 'hist',
    range = (1950, 2000),
    bins = 100,
    xticks=[x for x in range(1950, 1990, 10)],
    rot = -45,
    figsize = (12,6)
);
plt.title ('Распределение признака registration_year 1950-1990 года');

In [ ]:
data[['registration_year']].plot(
    kind = 'hist',
    range = (1990, 2005),
    bins = 30,
    xticks=[x for x in range(1990, 2005, 1)],
    rot = -45,
    figsize = (12,6)
);
plt.title ('Распределение признака registration_year 1990-2000 года');

In [ ]:
data['registration_year'].quantile([.03, .04, .05,.1,.50, .7, .95, .98]).to_frame().transpose()

Оставляем объекты зарегистрированные в период с 1990 г по 2017.

In [ ]:
data = data.loc[(data['registration_year']>1990) & (data['registration_year']<=2017)]
data.shape[0]

In [ ]:
data[['registration_year']].plot(kind = 'hist',
                                 bins = 30,
                                 rot = -45,
                                 xticks = [x for x in range(1990, 2017, 5)],
                                 grid = True)
data['registration_year'].quantile([.03, .04, .05,.1,.50, .7, .95, .98]).to_frame().transpose()

####  price

In [ ]:
data.boxplot(column=['price'],
             figsize=(12, 2),
             vert=False,
             patch_artist=True,
             boxprops=dict(facecolor='lightblue'),
             whiskerprops=dict(color='gray'),
             medianprops=dict(color='darkblue'),
             capprops=dict(color='gray')).set_title('Boxplot of price',
                                                    fontsize=14);
# Посмотрим на распределение
data['price'].describe().to_frame().transpose()

Мы видим, что присутствуют объекты с нулевой стоимостью, это явные аномалии.
Для стоимости автомобиля такой разбег по стоимости вполне допустим при условии что остальные признаки не противоречат, например высокая стоимость для автомобиля с большим пробегом, возрастом и марки "дешевого" ценового сегмента. И здесь сложнее сделать вывод по выбросам, так как очевидно, что автомобили премиум сегмента и очень старые автомобили (которые продают на запчасти или подросткам) будут сильно различаться в цене.


In [ ]:
data['price'].quantile([.02, .03, .04, .05,.1,.50, .7, .95, .98]).to_frame().transpose()


In [ ]:
data.loc[data['price']==0].shape[0]

Мы получаем, что 2.67% данных это объекты с нулевой стоимостью.
Их определенно нужно удалить. Иначе наши предсказания с высокой вероятностью будут иметь большее число отрицательных значений, что невозможно и потребуется более тщательная постобработка предсказаний.

In [ ]:
data = data.loc[data['price']>0]
data.head()

In [ ]:
data.boxplot(column=['price'],
             figsize=(12, 2),
             vert=False,
             patch_artist=True,
             boxprops=dict(facecolor='lightblue'),
             whiskerprops=dict(color='gray'),
             medianprops=dict(color='darkblue'),
             capprops=dict(color='gray')).set_title('Boxplot of price',
                                                    fontsize=14);
data['price'].quantile([.001, .05,.1,.50, .7, .95, .98]).to_frame().transpose()

Избавимся от редких значений: оставим только объекты с ценой выше 200 евро, что соответствует 5 квантилю.

In [ ]:
data = data.loc[data['price']>200]
data.head()

In [ ]:
data.boxplot(column=['price'],
             figsize=(12, 2),
             vert=False,
             patch_artist=True,
             boxprops=dict(facecolor='lightblue'),
             whiskerprops=dict(color='gray'),
             medianprops=dict(color='darkblue'),
             capprops=dict(color='gray')).set_title('Boxplot of price',
                                                    fontsize=14);
data['price'].quantile([.001, .05,.1,.50, .7, .95, .98]).to_frame().transpose()

In [ ]:
data['price'].quantile([.001, .05,.1,.50, .7, .95, .98]).to_frame().transpose()

In [ ]:
data[['price']].describe()

После удаления объектов с аномально низкой и редким значением стоимости мы видим, что:
-  средняя цена выросла с 4437 до 4670
- медианная стоимость незначительно упала с 2750 до 2700.0

Параметры выборки изменились в допустимых размерах.

Проанализируем объекты хвостовой части. Их мы рассмотрим совместно с пробегом и датой регистрацией автомобиля, так как большой пробег для высокой цены нелогичен.

In [ ]:
sns.heatmap(data.loc[data['price']>15000, ['price','kilometer','registration_year']]\
                              .sort_values(
                                          by = ['price','kilometer'],
                                          ascending=False)\
                               .corr());


Эти объекты действительно похожи на аномальные, так как не видно четкой зависимости между этими параметрами, тогда как в реальной жизни будет существовать зависимость:
- старше автомобиль- больше пробег
- больше пробег - ниже стоимость.
Если зависимость между годом регистрации и пробегом мы наблюдаем (хоть и не убедительную), то зависимость цены от этих параметров мы не наблюдаем вовсе.



In [ ]:
data.loc[data['price']>15000, 'kilometer'].value_counts().to_frame().transpose()

In [ ]:
data.loc[data['price']>15000, 'registration_year'].value_counts().sort_values(ascending = False).to_frame().head(15).transpose()

Посмотри на признак date_crawled (дата скачивания анкеты), чтобы понять когда сервис начал продавать автомобили

In [ ]:
data.loc[data['price']>15000].groupby('registration_year')[['price']].count().plot(kind = 'bar',
                                                        rot = 60,
                                                        color = 'green');

In [ ]:
pd.to_datetime(data['date_crawled']).dt.year.value_counts()


На этом графике мы видим, что объекты стоимостью > 15 тыс присутствуют во всех возрастных категориях, без исключений, что неправдоподобно.
Еще одна странная особенность, что наибольшее число дорогих объектов находятся в диапазоне 2007-2014 год, что странно, учитывая что все анкеты скачаны не позднее 2016 г, что говорит о том, что дата продажи не ранее 2016 г.
Значит объекты зарегистрированные в 2007 году к моменту продажи будут очень старые для такой стоимости.

Удалим объекты для которых справедливы 2 условия:
- цена выше 15 тыс
- registration_year < 2014

In [ ]:
data.loc[(data['price']>=15000) & (data['registration_year']>2014)].shape[0]

In [ ]:
data.loc[data['price']>15000].groupby('registration_year')[['price']].count().plot(kind = 'bar',
                                                        rot = 60,
                                                        color = 'green');

In [ ]:
index_to_drop = data[(data['price'] >= 15000) & (data['registration_year'] < 2014)].index
index_to_drop

In [ ]:
data.drop(index = index_to_drop, inplace=True)

In [ ]:
data.loc[(data['price']>=15000) & (data['kilometer']>100000), 'brand']\
              .value_counts(ascending = True)\
              .plot( kind = 'barh',
                    color = 'brown');

In [ ]:
data.loc[(data['price']>=15000) & (data['kilometer']>100000)]\
                  .groupby('registration_year')[['price']]\
                  .count().plot(kind = 'bar',
                                rot = 60,
                                color = 'green');

In [ ]:
sns.heatmap(data[['price','kilometer','registration_year']].corr())

In [ ]:
data['price'].quantile([.001, .05,.1,.50, .7, .95, .98]).to_frame().transpose()

Теперь карта дешевых объектов не противоречит логике:
- высокая стоимость характерна для автомобилей премиум сегмента
- количество дорогих объектов (несмотря на высокий пробег) увеличивается по мере уменьшения возраста.

#### kilometer

In [ ]:
data[['kilometer']].hist(figsize=(10, 4));



In [ ]:
# гистограмма распределения количества автомобилей с пробегом менее 100 тыс
data.loc[data['kilometer']<=100000, 'kilometer'].hist(figsize=(12, 2))
plt.xticks([j for j in range(5000, 110000, 5000)],
           rotation=65)
plt.title('Количество автомобилей с пробегом менее 100 (тыс. км.)')
plt.xlabel('Пробег (км)')
plt.ylabel('Частота')
plt.show()


In [ ]:
# гистограмма количества автомобилей с пробегом от 100 тыс
data.loc[data['kilometer']>=100000, 'kilometer'].hist(figsize=(12, 2))
plt.xticks([j for j in range(100000, 160000, 5000)],
           rotation=65)
plt.title('Количество автомобилей с пробегом более 100 (тыс. км.)')
plt.xlabel('Пробег (км)')
plt.ylabel('Частота')
plt.show()


Гистограмма распределения объектов с большим пробегом выглядит очень странно:
- данные как будто сгруппированны по каждым 20 км и гистограмма имеет прерывистый вид
- тогда как гистограмма объектов с пробегом до 100 тыс непрерывна.

Это наводит меня на мысль, что после 100 тыс появилились синтетически в базе(возможно с другого агрегатора).

Тем не менее, обработать аномалии не получится, так как они сгруппированы по категориям уже, и сейчас данные выглядят реалистичными.

####power

При анализе этого параметра будем исходить из того, что минимальная мощность это 20 л.с а максимальная 1000.

In [ ]:
data.loc[(data['power']<=20) | (data['power']>1000)]


Очень большое количество аномальных значений которые удалять мы не будем из соображений сохранения данных.

Все объекты мы разделим на несколько категорий в зависимости от мощности.

- **1** группа: до 100 л.с.  Микроавтомобили и компактные автомобили, обычно это небольшие городские автомобили
-  **2** группа: от 100 до 200 л.с.   Это типичные легковые автомобили для повседневного использования.

- **3** группа: от 200 до 300 л.с. Бизнес-класс и премиум-автомобили

- **4** группа: от 300 л.с. до 400 л.с. Спортивные автомобили

- **5** группа: от 400 л.с. и выше. Это автомобили с экстремальной производительностью и инновационными технологиями.

Обратим внимание на автомобили с мощностью более 200. Если они не соответствуют по марке и стоимости премиум сегменту то заменим мощность на более низкую, так как вероятнее всего это ошибка.

Выбирать будем только по условию минимальной цены - цена для автомобилей с мощностью более 200 л.с. должна быть выше 5 тыс.

In [ ]:
data_to_plot = data.loc[data['power'] > 200, 'price']
plt.hist(data_to_plot,
         bins=30,
         edgecolor='black')
plt.xlabel('Стоимость')
plt.xticks(range(0, 21000, 1000),
           rotation=75)
plt.ylabel('Частота')
plt.title('Гистограмма стоимости автомобилей с мощностью > 200')
plt.show()

In [ ]:
data.loc[(data['power'] > 200) & (data['price'] < 5000)].shape[0]

In [ ]:
data.loc[data['power'] <= 100, 'power_category'] = 1
data.loc[
    (100 < data['power'])
    & (data['power']<= 200), 'power_category'] = 2

# для высокомощных объектов добавляем цену в условие
data.loc[
    (200 < data['power'])
    & (data['power']<= 300)
    & (data['price'] >= 5000), 'power_category'] = 3
data.loc[
    (200 < data['power'])
    & (data['power']<= 300)
    & (data['price'] < 5000),'power_category'] = 2

data.loc[
    (300 < data['power'])
    & (data['power']<= 400)
    & (data['price'] >= 5000), 'power_category'] = 4
data.loc[
    (300 < data['power'])
    & (data['power']<= 400)
    & (data['price'] < 5000), 'power_category'] = 2

data.loc[
    (400 < data['power'])
    & (data['power']<= 500)
    & (data['price'] >= 5000), 'power_category'] = 5
data.loc[
    (400 < data['power'])
    & (data['power']<= 500)
    & (data['price'] < 5000), 'power_category'] = 2

data.loc[
    (data['power'] > 500)
    & (data['price'] >= 5000),'power_category'] = 6
data.loc[
    (data['power'] > 500)
    & (data['price'] < 5000),'power_category'] = 2

Проверим не осталось ли объектов с мощностью более 200 при низкой стоимости

In [ ]:
data.query('price< 5000')[['power_category']].value_counts()

In [ ]:
# удалим признак power
data = data.drop('power', axis = 1)

#### pictures_amount

In [ ]:
data['pictures_amount'].unique()

Как видим этот признак неинформативен,  так как не содержит никакой информации и одинаков для всех объектов.

In [ ]:
data = data.drop( 'pictures_amount', axis=1)
data.head()

### Анализ категориальных признаков

In [ ]:
for column in categorical_columns:
    print (column, data[column].nunique())
    print ()

Уберем из этих признаков дату и модель автомобилей, чтобы удобно было визуализировать.

In [ ]:
categorical_columns

In [ ]:
categorical_columns = ['vehicle_type', 'gear_box',  'fuel_type', 'repaired']

In [ ]:
data[categorical_columns].head()

Визуализируем уникальные значения для каждого признака

In [ ]:
num_plots = len(categorical_columns)
fig, axes = plt.subplots(1, num_plots, figsize=(12, 5))

for i, col in enumerate(categorical_columns):
    ax = axes[i]
    sns.countplot(data=data, x=col, ax=ax)
    ax.set_title(f'Value Counts for {col}')

    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

plt.tight_layout()

plt.show()

In [ ]:
data['brand'].value_counts().sort_values(ascending = False).head(15)\
              .plot(kind = 'barh',
                    rot = 6,
                    color = 'orchid')
plt.title('Топ 15 самых популярных моделей')
plt.show()

In [ ]:
#проверим на неявные дупликаты
data['brand'].unique()

In [ ]:
data['model'].unique()

In [ ]:
for brand in data['brand'].unique():
    print(brand,
          data.loc[data['brand']==brand, 'model'].nunique(),
          data.loc[data['brand']==brand, 'model'].unique())
    print()

неявных дупликатов не обнаружила

### Признаки с датой

In [ ]:
data.columns

In [ ]:
date_columns = ['date_crawled', 'date_created','registration_year', 'registration_month', 'last_seen']

In [ ]:
data[date_columns].head()

Выше мы сравнили признаки date_crawled и date_created. Мы добавили новый синтетический признак 'date_difference', который содержит разницу с момента скачивания до момента создания анкеты. Поэтому удалим признак date_crawled.


In [ ]:
data = data.drop('date_crawled', axis =1)

In [ ]:
data.shape[0]

In [ ]:
# сравним date_created и last_seen.
period = (data['last_seen'] - data['date_created']).dt.days

In [ ]:
period.describe().to_frame()

Неясно что происходит с анкетой после продажи или срок дезактивации, но можно предположить что средний срок продажи для большинства пользователей это 14 дней.


In [ ]:
date_columns = ['date_created', 'registration_year', 'registration_month', 'last_seen']

In [ ]:
data[date_columns].head()

Добавим новый признак разницу между date_created и last_seen, чтобы удалить признак last_seen и оставить у признака date_created месяц и год.

In [ ]:
data['lastseen_days'] = period
data['lastseen_days']

In [ ]:
data = data.drop('last_seen', axis = 1)

In [ ]:
data['created_year'] = pd.to_datetime(data['date_created']).dt.year

In [ ]:
data['created_month'] = pd.to_datetime(data['date_created']).dt.month

In [ ]:
data = data.drop('date_created', axis = 1)

In [ ]:
data.head()

Добавим новый признак - возраст автомобиля

In [ ]:
data['avto_age'] = data['created_year'] - data['registration_year']
data['avto_age'].head()

In [ ]:
data.head()

Дубликатов не наблюдается.

# Модели

### Делим выборки

In [ ]:
#Разделим выборки
train, validation = train_test_split(data,
                                     random_state=21,
                                     test_size = 0.4)
validation, test = train_test_split(validation,
                                    random_state=21,
                                    test_size = 0.5)
print('train', train.shape[0])
print('validation', validation.shape[0])
print('test', test.shape[0])

In [ ]:
#выделим обучающие признаки и целевой признак
features = data.drop('price', axis = 1)
target = data['price']
data.columns
X = list(features.columns)
y = ['price']

In [ ]:
categorical = list(features.select_dtypes(include = ['object', 'bool']).columns)
numeric = list(features.select_dtypes(include = ['number']).columns)
print('categorical', categorical)
print()
print('numeric', numeric)

## LinearRegression и RandomForestRegressor

**Обоснование выбора модели**

Согласно заданию нам необходимо обязательно использовать простые модели (не бустинг), поэтому я решила использовать две модели библиотеки Sclearn:
- LinearRegression
- RandomForestRegressor.

Я выбрала эти модели по следующим соображениям:
- **LinearRegression** работают быстрее, а учитывая что для нашей работы время обучения и предсказания являются важными метриками, то я решила начать с самой быстрой модели, которая может дать необходимую метрику `RMSE <2500`.

- RandomForestRegressor отличается высокой точностью предсказаний на дефолтных параметрах и имеется возможность подобрать гиперпараметры, в отличии от линейной регрессии.

Далее я буду использовать алгоритмы градиентного бустинга:

- CatBoostRegressor
- LightGBM.

Для всех моделей я буду делать подбор гиперпараметров методом **RandomisedGridsearchCV**.

Выбор этого метода подбора так же продиктован ограниченностью вычислительных ресурсов, т.к. **GridsearchCV** перебирает большее число параметров, а значит увеличивается время на работу данного метода.
**Optuna** крайне ресурсозатратна, и при таком низком пороге метрики ее использование нецелесообразно.




#### Предобработка признаков

Наши данные содержат категориальные и числовые признаки, которые нужно обрабатывать прежде чем можно передать их в выбранные нами модели машинного обучения.

Поэтому мы будем использовать **pipeline** для предобработки и для обучения.
pipeline предобработки включает в себя следующие этапы:

**Трансформер для числовых признаков**:
- масштабирование StandartScaler
- анализ важности признаков PCA(Principal Component Analysis)

**Трансформер для категориальных признаков**:
- кодирование с помощью OneHotEncoder

**ColumnTransformer** для объединения результатов трансформаций в одну матрицу признаков


In [ ]:
numeric_transformer = Pipeline(
    steps =[
        ('scl', StandardScaler()),
         ('pca', PCA(n_components = 0.95))
    ])

In [ ]:
categorical_transformer = Pipeline(
    steps = [('enc', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
preprocessor = ColumnTransformer( transformers=[
        ("num", numeric_transformer, numeric),
        ("cat", categorical_transformer, categorical),
    ])

####LinearRegression_pipeline

In [ ]:
lr_pipe=Pipeline(steps = [('preprocess', preprocessor),
 ('clf',LinearRegression(random_state=42))])

In [ ]:
start = time.time()
lr_pipe.fit(train, train[y])
end = time.time()
lr_fit_time = end - start
print(lr_fit_time, 'сек')

In [ ]:
start = time.time()
predict_lr = lr_pipe.predict(validation)
end = time.time()
lr_pred_time = end - start
print(lr_pred_time, 'сек' )

In [ ]:
rmse_lr = mean_squared_error(validation[y], predict_lr, squared=False)
mape_lr = mean_absolute_percentage_error(validation[y], predict_lr)

print(f'RMSE LinearRegression: {rmse_lr:.3f}')
print(f'MAPE LinearRegression: {mape_lr:.3f}%')

####RandomForestRegressor_pipeline

In [ ]:
rand_forest_pipe=Pipeline(steps = [('preprocess', preprocessor),
 ('rand_forest', RandomForestRegressor(random_state=42))])

In [ ]:
start = time.time()

rand_forest_pipe.fit(train, train[y])

end = time.time()
rand_forest_fit_time = end - start
print(rand_forest_fit_time, 'сек')

In [ ]:
params = rand_forest_pipe.named_steps['rand_forest'].get_params()
params

In [ ]:
start = time.time()

predict_rand_forest = rand_forest_pipe.predict(validation)

end = time.time()
rand_forest_predict_time = end - start
print(rand_forest_predict_time, 'сек')

In [ ]:
rmse_rand_forest = mean_squared_error(validation[y], predict_rand_forest, squared=False)
mape_rand_forest = mean_absolute_percentage_error(validation[y], predict_rand_forest)

print(f'RMSE RandomForestRegressor: {rmse_rand_forest:.3f}')
print(f'MAPE RandomForestRegressor: {mape_rand_forest:.3f}%')

In [ ]:
index = ['linear_regression', 'random_forest_regressor']
columns = ['RMSE', 'MAPE', 'fit_time (сек)', 'predict_time (сек)']
data = [[mean_squared_error(validation[y], predict_lr, squared=False),
        mean_absolute_percentage_error(validation[y], predict_lr),
         lr_fit_time,
         lr_pred_time],
        [mean_squared_error(validation[y], predict_rand_forest, squared=False),
        mean_absolute_percentage_error(validation[y], predict_rand_forest),
         rand_forest_fit_time,
         rand_forest_predict_time]]

In [ ]:
result = pd.DataFrame(data = data,
             index = index,
             columns = columns)
result

Таким образом мы получили требуемый показатель метрики MSE < 2500.


Добавим перебор параметров для модели случайного леса

In [ ]:
param_grid = (
    {'n_estimators': [100, 200, 300],
     'max_depth': [10, 20, 30],
     'min_samples_split': [2, 5, 10],
     'min_samples_leaf': [1, 2, 4]})

In [ ]:
best_model = None
estimator= RandomForestRegressor(random_state=42)

In [ ]:
grid_search = GridSearchCV(estimator = estimator,
                           param_grid = param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error')

In [ ]:
rgscv_pipe = Pipeline(steps = [('prepp', preprocessor),
                               ('grid_search', grid_search)])

In [ ]:
start = time.time()

rgscv_pipe.fit(train, train[y])

end = time.time()
rand_search_fit_time = end - start
print(rand_search_fit_time, 'сек')

In [ ]:
best_model = rgscv_pipe.named_steps['grid_search'].best_estimator_
best_model

In [ ]:
best_model_params = best_model.get_params()
best_model_params

In [ ]:
start = time.time()

rand_search_predict = rgscv_pipe.predict(validation)

end = time.time()
rand_search_predict_time = end - start
print(rand_search_predict_time, 'сек')

In [ ]:
mse_best_parameters = mean_squared_error(validation[y], rand_search_predict)
mape_best_parameters = mean_absolute_percentage_error(validation[y], rand_search_predict)
print(f'RMSE RandomForestRegressor (best_parameters): {rmse_best_parameters :.3f}')
print(f'MAPE RandomForestRegressor (best_parameters): {mape_best_parameters :.3f}%')

## Модели градиентного бустинга

### СatBoost

Для СatBoost мы будем использовать некодированные данные.

In [ ]:
categorical

In [ ]:
cat_features = categorical

In [ ]:
train_pool = Pool(data=train[X],
                  label=train[y],
                  cat_features=cat_features)
print(train_pool.shape[0])

In [ ]:
validation_pool = Pool(data=validation[X],
                       label=validation[y],
                  cat_features=cat_features)
print(validation_pool.shape[0])

In [ ]:
test_pool = Pool(data=test[X],
                  label=test[y],
                  cat_features=cat_features)
print(test_pool.shape[0])

#### RandomGridSearch

In [ ]:
# параметры для перебора
param_grid = {
    'learning_rate': [0.001, 0.10],
    'depth': [6, 15],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'early_stopping_rounds' = 20
}

In [ ]:
params = {
    'cat_features': cat_features,
    'eval_metric': 'MAPE',
    'loss_function': 'RMSE',
    'random_seed': 42,
    'verbose': 2,
    'task_type':'GPU'
    }

In [ ]:
#инициализируем модель
model = CatBoostRegressor(**params)

In [ ]:
now = datetime.datetime.now()

print(now)

In [ ]:
%%time
randomized_search_result = model.randomized_search(param_grid,
                                                   train_pool)
print (model.best_score_)

In [ ]:
end = datetime.datetime.now()
catboost_gscv_time = end-now
print('catboost_gscv_time', catboost_gscv_time)

In [ ]:
best_params = randomized_search_result['params']
best_params

In [ ]:
# оцениваем
y_pred = model.predict(validation_pool)

In [ ]:
rmse = mean_squared_error(validation[y], y_pred, squared=False)

#  MAPE
mape = mean_absolute_percentage_error(validation[y], y_pred)

print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.3f}%")

In [ ]:
fstrs = model.get_feature_importance(prettified=True)

fig, ax = plt.subplots()
fstrs.plot(x='Feature Id', y='Importances', kind='bar', ax=ax)
ax.set_title("Важность признаков")
ax.set_ylabel("Значение")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
best_model = CatBoostRegressor(**best_params,
                               random_seed= 42,
                              verbose = 100,
                              iterations = 1600)

In [ ]:
%%time
# обучаем модель
best_model.fit(test_pool)
print (best_model.best_score_)

In [ ]:
pred_test = model.predict(test[X])
pred_test

In [ ]:
#pred_test.loc[prediction['sellingprice']<250, 'sellingprice']=250
#pred_test.loc[prediction['sellingprice']<250].shape[0]

### Lightgbm

In [ ]:
!pip install lightgbm
import lightgbm as lgb

In [ ]:
param['metric'] = ['mape', 'rmse']

In [ ]:
num_round = 10
lgb_model = lgb.train(param,
                train,
                num_round,
                nfold=5)

In [ ]:
lgb_pred = lgb_model.predict(validation,
                             num_iteration = lgb_model.best_iteration
                             )